In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv
/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00005.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00057.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00020.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00072.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00040.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00043.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00010.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00071.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/image

In [2]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy as sp
import soundfile as sf

# Loading Select Data

In [3]:
# There are 10 different Genres in the GTZAN Music Genre Dataset
# Let us load the first example wav file of each
blues_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00000.wav"
classical_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/classical.00000.wav"
country_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/country/country.00000.wav"
disco_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00000.wav"
hiphop_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/hiphop/hiphop.00000.wav"
jazz_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00000.wav"
metal_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/metal/metal.00000.wav"
pop_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/pop/pop.00000.wav"
reggae_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/reggae/reggae.00000.wav"
rock_file = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/rock/rock.00000.wav"

In [ ]:
# we can play any file as :
ipd.Audio(rock_file)

These files also have vocals in them, before we try the FT on these audios, lets seperate the vocals out of the audio files. We can do this using librosa

In [4]:
# loading the files:
# the sample rate sr remains the same, so lets only take it once
blues, sr = librosa.load(blues_file)
classical, _ = librosa.load(classical_file)
country, _ = librosa.load(country_file)
disco, _ = librosa.load(disco_file)
hiphop, _ = librosa.load(hiphop_file)
jazz, _ = librosa.load(jazz_file)
metal, _ = librosa.load(metal_file)
pop, _ = librosa.load(pop_file)
reggae, _ = librosa.load(reggae_file)
rock, _ = librosa.load(rock_file)

In [5]:
# a function to seperate vocals:
def seperate_vocals(audio):
    # get the magnitude and phase from Short Term FT
    S_full, phase = librosa.magphase(librosa.stft(audio))
    
    # from the librosa documentation:
    
    # We'll compare frames using cosine similarity, and aggregate similar frames
    # by taking their (per-frequency) median value.
    #
    # To avoid being biased by local continuity, we constrain similar frames to be
    # separated by at least 2 seconds.
    #
    # This suppresses sparse/non-repetetitive deviations from the average spectrum,
    # and works well to discard vocal elements.

    S_filter = librosa.decompose.nn_filter(S_full,
                                           aggregate=np.median,
                                           metric='cosine',
                                           width=int(librosa.time_to_frames(2, sr=sr)))

    # The output of the filter shouldn't be greater than the input
    # if we assume signals are additive.  Taking the pointwise minimium
    # with the input spectrum forces this.
    S_filter = np.minimum(S_full, S_filter)
    
    # We can also use a margin to reduce bleed between the vocals and instrumentation masks.
    # Note: the margins need not be equal for foreground and background separation
    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,
                                   margin_i * (S_full - S_filter),
                                   power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,
                                   margin_v * S_filter,
                                   power=power)

    # Once we have the masks, simply multiply them with the input spectrum
    # to separate the components

    S_foreground = mask_v * S_full
    S_background = mask_i * S_full
    
    return S_foreground, S_background

In [6]:
# getting instrumental and vocals seperated for the audios:
blue_i, blue_v = seperate_vocals(blues)

In [14]:
# saving the files
import soundfile as sf
sf.write('blue_i.wav', blue_i, sr, subtype='PCM_24')

RuntimeError: Error opening 'blue_i.wav': Format not recognised.